In [1]:
pip install lightning pytorch-forecasting pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 1.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [102]:
import pandas as pd
import lightning.pytorch as pl
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer, QuantileLoss
from lightning.pytorch.tuner import Tuner
import seaborn as sns
from datetime import datetime
import numpy as np

# Load Dataset

In [83]:
data = sns.load_dataset('flights')

# Crea una nuova colonna datetime combinando 'year' e 'month'
data['date'] = pd.to_datetime(data.apply(lambda x: f"{x.year}-{x.month}-01", axis=1))

# Aggiungi un ID univoco per la serie temporale
data['group'] = "AirPassengers"

# Aggiungi un time_idx per indicizzare il tempo
# Crea un nuovo time_idx continuo
data['time_idx'] = (data['date'] - data['date'].min()).dt.days // 30
data['time_idx'] = data.groupby('group').cumcount()

data.passengers = data.passengers.astype('float64')

In [84]:
data.head()

,year,month,passengers,date,group,time_idx
0,1949,Jan,112.0,1949-01-01,AirPassengers,0
1,1949,Feb,118.0,1949-02-01,AirPassengers,1
2,1949,Mar,132.0,1949-03-01,AirPassengers,2
3,1949,Apr,129.0,1949-04-01,AirPassengers,3
4,1949,May,121.0,1949-05-01,AirPassengers,4


In [85]:
data['time_idx'].unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143])

In [86]:
data[['time_idx', 'passengers']].head()

,time_idx,passengers
0,0,112.0
1,1,118.0
2,2,132.0
3,3,129.0
4,4,121.0


In [87]:
print(data['time_idx'].unique())
print(data['time_idx'].isnull().sum())

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143]
0


In [88]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   year        144 non-null    int64         
 1   month       144 non-null    category      
 2   passengers  144 non-null    float64       
 3   date        144 non-null    datetime64[ns]
 4   group       144 non-null    object        
 5   time_idx    144 non-null    int64         
dtypes: category(1), datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 6.3+ KB


In [89]:
data.describe()

,year,passengers,date,time_idx
count,144.000000,144.000000,144,144.000000
mean,1954.500000,280.298611,1954-12-16 05:00:00,71.500000
min,1949.000000,104.000000,1949-01-01 00:00:00,0.000000
25%,1951.750000,180.000000,1951-12-24 06:00:00,35.750000
50%,1954.500000,265.500000,1954-12-16 12:00:00,71.500000
75%,1957.250000,360.500000,1957-12-08 18:00:00,107.250000
max,1960.000000,622.000000,1960-12-01 00:00:00,143.000000
std,3.464102,119.966317,NaN,41.713307


In [90]:
data.groupby('year').sum('passengers')

,passengers,time_idx
year,,
1949,1520.0,66
1950,1676.0,210
1951,2042.0,354
1952,2364.0,498
1953,2700.0,642
1954,2867.0,786
1955,3408.0,930
1956,3939.0,1074
1957,4421.0,1218


# Define Trainer

### Spiegazione dei Parametri e Passaggi

- **`data[lambda x: x.date <= training_cutoff]`**:
  - **Descrizione**: Filtro per includere solo i dati fino alla data di cutoff (`1958-12-01`) per l'addestramento. Questo parametro rappresenta il subset del dataset che sarà utilizzato per il training.

- **`time_idx="time_idx"`**:
  - **Descrizione**: Nome della colonna che rappresenta l'indice temporale. È usato per comprendere l'ordine temporale degli eventi nella serie temporale.

- **`target="passengers"`**:
  - **Descrizione**: Nome della colonna contenente il valore da prevedere. In questo caso, il numero di passeggeri.

- **`group_ids=["group"]`**:
  - **Descrizione**: Nome della colonna che identifica le serie temporali uniche. Utile per trattare separatamente più serie temporali nel dataset.

- **`max_encoder_length=max_encoder_length`**:
  - **Descrizione**: Lunghezza massima della sequenza di input. Indica quanti passi temporali precedenti il modello considera per fare una previsione (36 mesi nel tuo caso).

- **`max_prediction_length=max_prediction_length`**:
  - **Descrizione**: Lunghezza massima della sequenza di output. Indica quanti passi temporali futuri il modello prevede in una volta (6 mesi nel tuo caso).

- **`time_varying_known_reals=["time_idx"]`**:
  - **Descrizione**: Colonne contenenti variabili reali conosciute nel futuro, che possono influenzare le previsioni. Qui, `time_idx` è una variabile nota in anticipo.

- **`time_varying_unknown_reals=["passengers"]`**:
  - **Descrizione**: Colonne contenenti variabili reali non conosciute in anticipo, che il modello deve prevedere. In questo caso, il numero di passeggeri da predire.

- **`validation`**:
  - **Descrizione**: Crea un dataset di validazione basato sul dataset di training, utilizzando dati successivi alla data di cutoff per testare le prestazioni del modello.

- **`train_dataloader` e `val_dataloader`**:
  - **Descrizione**: Creano i `DataLoader` per il training e la validazione, che suddividono i dati in batch e gestiscono il caricamento parallelo (utilizzando `num_workers=2` per la parallelizzazione).

In [118]:
max_encoder_length = 12  # Il modello utilizza fino a 36 passi temporali precedenti (ad es., mesi) per fare una previsione.
max_prediction_length = 3  # Il modello prevede fino a 6 passi temporali futuri in una singola previsione.
training_cutoff = datetime(1958, 12, 1)  # I dati fino al 1 dicembre 1958 sono utilizzati per addestrare il modello. I dati successivi possono essere usati per la validazione o il test.

training = TimeSeriesDataSet(
    data[lambda x: x.date <= training_cutoff],
    time_idx="time_idx",
    target="passengers",
    group_ids=["group"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    time_varying_known_reals=["time_idx"],
    time_varying_unknown_reals=["passengers"]
)

validation = TimeSeriesDataSet.from_dataset(training, data, min_prediction_idx=training.index.time.max() + 1, stop_randomization=True)

batch_size = 32
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=2)
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=2)

# Define Model

In [120]:
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=1, verbose=False, mode="min")
lr_logger = LearningRateMonitor()

trainer = pl.Trainer(
    max_epochs=100,
    accelerator="auto",
    gradient_clip_val=0.1,
    limit_train_batches=30,
    callbacks=[lr_logger, early_stop_callback],
    logger=TensorBoardLogger("lightning_logs")
)

INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [121]:
tft = TemporalFusionTransformer.from_dataset(
    training,
    hidden_size=32,
    attention_head_size=1,
    dropout=0.1,
    hidden_continuous_size=16,
    loss=QuantileLoss(),
    log_interval=2,
    learning_rate=0.03,
    reduce_on_plateau_patience=4
)

tft.size()

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.


60197

In [97]:
res = Tuner(trainer).lr_find(
    tft, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader, early_stop_threshold=1000.0, max_lr=0.3,
)

fig = res.plot(show=True, suggest=True)
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [98]:
trainer.fit(
    tft, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader,
)

INFO: 
   | Name                               | Type                            | Params | Mode 
------------------------------------------------------------------------------------------------
0  | loss                               | QuantileLoss                    | 0      | train
1  | logging_metrics                    | ModuleList                      | 0      | train
2  | input_embeddings                   | MultiEmbedding                  | 0      | train
3  | prescalers                         | ModuleDict                      | 64     | train
4  | static_variable_selection          | VariableSelectionNetwork        | 0      | train
5  | encoder_variable_selection         | VariableSelectionNetwork        | 3.8 K  | train
6  | decoder_variable_selection         | VariableSelectionNetwork        | 1.8 K  | train
7  | static_context_variable_selection  | GatedResidualNetwork            | 4.3 K  | train
8  | static_context_initial_hidden_lstm | GatedResidualNetwork            | 4

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [124]:
from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import MAE, SMAPE, PoissonLoss, QuantileLoss

predictions = tft.predict(val_dataloader)

INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [129]:
print(type(predictions))
print(predictions.shape)

<class 'torch.Tensor'>
torch.Size([36, 3])
